In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import pandas as pd
import zipfile
import torch
import json

In [ ]:
# Path to your folder in Google Drive
drive_path = '/content/drive/My Drive/MA/'

In [ ]:
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU found. Please enable GPU in runtime settings.")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model = model.to(device)

GPU is available: Tesla T4


In [ ]:
# Unzipping files
with zipfile.ZipFile(drive_path + 'article_embeddings.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/article_embeddings')

In [ ]:
with zipfile.ZipFile(drive_path + 'message_embeddings.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/message_embeddings')

In [ ]:
# Load the .txt file as JSON
article_embeddings_path = '/content/article_embeddings/article_embeddings.txt'
message_embeddings_path = '/content/message_embeddings/message_embeddings.txt'

with open(article_embeddings_path, 'r') as file:
    article_data = json.load(file)

with open(message_embeddings_path, 'r') as file:
    article_data = json.load(file)

In [ ]:
domain_index = np.load(drive_path + 'domain_index.npy')
chat_index = np.load(drive_path + 'chat_index.npy')
url_domains = pd.read_csv(drive_path + 'url_domains.csv')

NameError: name 'np' is not defined

In [ ]:
# Ensure 'domain' is the correct column name in url_domains
grouped_domains = url_domains.groupby('domain_index').agg(list)

grouped_domains = grouped_domains.to(device)

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming `chat_index` maps to `url_domains`
train_indices, test_indices = train_test_split(range(len(chat_index)), test_size=0.2, random_state=42)

# Creating train-test splits
train_data = {
    'chat_index': chat_index[train_indices],
    'domain_index': domain_index[train_indices],
    'message_embeddings': message_embeddings[train_indices]
}

test_data = {
    'chat_index': chat_index[test_indices],
    'domain_index': domain_index[test_indices],
    'message_embeddings': message_embeddings[test_indices]
}


## Chat URL Matrix

In [ ]:
from scipy.sparse import csr_matrix

# Assuming `chat_index` and `domain_index` are integers for mapping
chat_url_matrix = csr_matrix(
    (np.ones(len(train_data['chat_index'])),
     (train_data['chat_index'], train_data['domain_index']))
)


## Features

In [ ]:
# Explicit variables: Message embeddings and domain index
explicit_features = np.hstack([message_embeddings, domain_index.reshape(-1, 1)])

# Implicit features: Interaction data from chat-url matrix
implicit_features = chat_url_matrix @ explicit_features

# Combine all features
gnn_features = np.hstack([explicit_features, implicit_features])


In [ ]:
np.save('/content/drive/My Drive/MA/gnn_features.npy', gnn_features)

Next Steps:
read following files  
*   article_embeddings.zip
*   message_embedding.zip
*   domain_index.npy
*   chat_index.npy
*   top_1000_urls.csv.gz
*   url_domains.csv
*   (url_data.parquet?)

group by domain

make train and test split

create features

train content GNN

train content agnostic GNN

train URL based GNN

